# Roast My Spotify Playlist

August 2024 was all about 'demure' and asking ChatGPT to roast your instagram. 

I decided to participate in this roasting trend but with my Spotify Playlist!

# Setup

In [1]:
import os 
import base64
from requests import post, get
import json
import pandas as pd
import os
from dotenv import load_dotenv

#### This allows you to read the API keys from your .env file

In [2]:
# Load the environment variables from .env
load_dotenv()

True

# 1a. Access the Spotify API

This uses the `spotipy` library to authenticate and access a particular user's profile. 

It will load a Spotify Login Page asking the user to login and approve access. 

This was a bit fiddly for me; I had to copy the resulting URL and paste it back into this notebook. 

If successful it will print all your playlists

In [ ]:
#!pip install spotipy

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Define your Spotify application credentials
SPOTIPY_CLIENT_ID = os.environ.get("SPOTIFY_CLIENT_ID")
SPOTIPY_CLIENT_SECRET = os.environ.get("SPOTIFY_CLIENT_SECRET")
SPOTIPY_REDIRECT_URI = 'http://localhost/'
SCOPE = 'playlist-read-private'

# Set up Spotify authentication
sp_oauth = SpotifyOAuth(
    client_id=SPOTIPY_CLIENT_ID,
    client_secret=SPOTIPY_CLIENT_SECRET,
    redirect_uri=SPOTIPY_REDIRECT_URI,
    scope=SCOPE
)

# Get the access token
token_info = sp_oauth.get_access_token(as_dict=False)

if token_info:
    sp = spotipy.Spotify(auth=token_info)
    print("Authenticated successfully!")

    # Fetch & Print the current user's playlists
    # you'll need the playlist_id later
    playlists = sp.current_user_playlists()
    print("User's Playlists:")
    for playlist in playlists['items']:
        print(f"Playlist: {playlist['name']}, ID: {playlist['id']}")
else:
    print("Failed to authenticate.")

### 1b. Function to get one of the above playlists

In [76]:
def get_playlist_tracks(username,playlist_id):

    #initialise an empty playlist
    playlist=[]
    
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])

    for track in tracks:
        trackname = track['track']['name']
        artist = track['track']['artists'][0]['name']
        #print(trackname,"-", artist)
        playlist.append("'{}' by {}".format(trackname, artist))

    playlist.reverse() #reverse chronological order

    print("Complete. Playlist has {} songs".format(len(playlist)))
    return playlist



Complete. Playlist has 77 songs


### 1c. Call the function `get_playlist_tracks()` 

Pass in your `userid` and `playlistid`

In [ ]:
playlist = get_playlist_tracks("liampearson", "0YynxlZ4jDaT8QOSA8RJfT")

In [70]:
# the variable roast_this_playlist converts the songs into one string
# this makes it easier for the LLM to process each of the different songs
roast_this_playlist = "\n".join(playlist)
print(roast_this_playlist)

'Diet Pepsi' by Addison Rae
'Bright Lights' by The Killers
'Guess featuring Billie Eilish' by Charli xcx
'Nobody’s' by Touché Amoré
'Angel Of My Dreams' by JADE
'JOYRIDE' by Kesha
'Falling Forever - Extended' by Dua Lipa
'911 - Charli XCX & A. G. Cook Remix' by Lady Gaga
'Girl, so confusing featuring lorde' by Charli xcx
'Your Light' by The Big Moon
'Close To You' by Gracie Abrams
'Hello goodbye' by Charli xcx
'Girl, so confusing' by Charli xcx
'Cry' by Benson Boone
'Beautiful Things' by Benson Boone
'Want/Need' by Ceres
'L’AMOUR DE MA VIE [OVER NOW EXTENDED EDIT]' by Billie Eilish
'Risk' by Gracie Abrams
'Dizzy' by Olly Alexander (Years & Years)
'Britpop' by A. G. Cook
'ZORRA' by Nebulossa
'360' by Charli xcx
'Sun Don't Shine' by Klangkarussell
'Rim Tim Tagi Dim' by Baby Lasagna
'The Tower' by LUNA
'We Will Rave' by Kaleen
'la noia' by Angelina Mango
'Pedestal - Eurovision version' by Aiko
'Inside' by Pretty Girl
'Falling Forever' by Dua Lipa
'i like the way you kiss me' by Artemas
'H

# 2. Access the OpenAI API

In [ ]:
#!pip install openai

In [ ]:
from openai import OpenAI

#retrieve your OpenAI API secret key from your .env file
client = OpenAI(api_key=os.environ.get("OPENAI_SECRET_KEY"))

## 2a. The Prompt

In [85]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a music critic tasked with roasting my spotify playlist."},
        {
            "role": "user",
            "content": "Roast my Spotify playlist in 100 words or less: {}".format(roast_this_playlist)
        }
    ]
)

## 2b. The Roast (Response) and Cost

~70 songs cost 2cents!

In [90]:
print("RESPONSE:\n")
print(completion.choices[0].message.content)

#calculate cost of the prompt
cost_per_prompt_token = 15/1000000 # USD$0.15 per million tokens
cost_per_completion_token = 60/1000000 # USD$0.60 per million tokens

prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

cost =  (prompt_tokens*cost_per_prompt_token) + (completion_tokens*cost_per_completion_token)

print("\nCOST: ${} (USD)".format(round(cost,2)))

RESPONSE:

Ah, your playlist is like a chaotic mixtape made by a toddler after a sugar rush at a pop festival. You've got Addison Rae trying to be the next pop sensation next to The Killers, who are like, “What happened to our legacy?” Charli XCX is clearly auditioning for the role of 'Most Over-Featured Artist'—I lost count after the third track! And can we talk about the sheer randomness of “Rim Tim Tagi Dim” and “Voglio Vederti Danzare”? Do you get your song selections from a "What’s Trending Today?" app? Bless your heart for trying, but this is a slippery slope into music purgatory!

COST: $0.02 (USD)
